通常文章會 (1)太長塞不進去 LLM (2)資訊太雜導致不能產生一個有效代表的 embedding，因此我們常會需要將 Document 切成一個個的片段。雖然聽起來簡單但其實有很多學問。

在 Text Splitting 我們會想要達成以下的目標：
1. 片段足夠小
2. 片段內部高度意義相關
3. 高度意義相關的資訊盡量不要被切到不同片段

而此時我們要考慮的點是

1. 如何切分文字
2. 如何測量片段（Chunk）的大小

# 1. Recursive Splitting

這裡介紹一種官方建議，常用且通用的作法
1. 將文字切分成很小但高度意義相關的片段單位（通常是句子）
2. 將小片段重新接合成大片段直到達到設定的片段大小
3. 隨後在與剛做好的大片段重疊一部份的地方，重新開始收集小片段組合成大片段。讓片段之間重疊的目的是避免資訊在片段之間被攔腰斬斷。

In [1]:
from langchain.document_loaders import NotionDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_setup import pprint_documents

documents = NotionDirectoryLoader(path='../../data/notion').load()

char_based_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100, # 每個文件不可超過 100 字元
    chunk_overlap  = 20, # 每個文件從上一個文件的倒數第 20 個字元開始
    length_function = len,
    add_start_index = True,
)

split_documents = char_based_splitter.split_documents(documents=documents)

pprint_documents(split_documents)

Document 1:

今天來介紹日本最具代表性的軟體開發手法
它的名子為 **隕石落下型開發**。

# 第一節

Metadata:{'source': '..\\..\\data\\notion\\隕石落下式開發法.md', 'start_index': 0}
----------------------------------------------------------------------------------------------------
Document 2:

通常的**瀑布式開發**是像下面這樣的形式:
| 步驟 | 內容     | 負責人     |
| ---- | -------- | ---------- |

Metadata:{'source': '..\\..\\data\\notion\\隕石落下式開發法.md', 'start_index': 47}
----------------------------------------------------------------------------------------------------
Document 3:

| 1    | 要件定義 | Producer   |
| 2    | 基本設計 | Director   |
| 3    | 詳細設計 | Planner    |

Metadata:{'source': '..\\..\\data\\notion\\隕石落下式開發法.md', 'start_index': 129}
----------------------------------------------------------------------------------------------------
Document 4:

| 4    | 實裝     | Programmer |

Metadata:{'source': '..\\..\\data\\notion\\隕石落下式開發法.md', 'start_index': 216}
------------------------------------------------------------------------------------------

但是其實大語言模型 (LLM) 真正的限制輸入長度是以 "token" 為單位而非字元，"token" 是語言模型定義的最小處理單位，可能是一個詞 (word) 或部分的詞 (sub-word)，每個語言模型可能基於不同的辭典 (vocabulary) 且不同辭典不能互通。而 Langchain 也提供了現成的做法讓使用者能改以 "token" 為單位計算長度。

In [2]:
token_based_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-3.5-turbo-16k", # 使用該模型所使用的辭典 (Vocabulary)
    chunk_size = 100, # 每個文件不可超過 100 toknes
    chunk_overlap  = 20, # 每個文件從上一個文件的倒數第 20 個 token 開始
    add_start_index = True,
)

split_documents = token_based_splitter.split_documents(documents=documents)

pprint_documents(split_documents)

Document 1:

今天來介紹日本最具代表性的軟體開發手法
它的名子為 **隕石落下型開發**。

# 第一節

Metadata:{'source': '..\\..\\data\\notion\\隕石落下式開發法.md', 'start_index': 0}
----------------------------------------------------------------------------------------------------
Document 2:

通常的**瀑布式開發**是像下面這樣的形式:
| 步驟 | 內容     | 負責人     |
| ---- | -------- | ---------- |
| 1    | 要件定義 | Producer   |
| 2    | 基本設計 | Director   |

Metadata:{'source': '..\\..\\data\\notion\\隕石落下式開發法.md', 'start_index': 47}
----------------------------------------------------------------------------------------------------
Document 3:

| 2    | 基本設計 | Director   |
| 3    | 詳細設計 | Planner    |
| 4    | 實裝     | Programmer |

Metadata:{'source': '..\\..\\data\\notion\\隕石落下式開發法.md', 'start_index': 158}
----------------------------------------------------------------------------------------------------
Document 4:

而**隕石式開發**是像下面這樣子的形式：
|     | 步驟 | 內容     | 負責人     |
| --- | ---- | -------- | ---------- |
| 神  | 1    | 要件定義 | Producer   |

Metadata:{'